In [1]:
import numpy as np,pandas as pd,json,sqlite3,requests, os
from jsonpath_ng import jsonpath
from jsonpath_ng.ext import parse
from pandas.io.json import json_normalize
from dotenv import load_dotenv
load_dotenv()
db_path = os.environ.get('dsa_db')
char_path = os.environ.get('char_data')
game_path = os.environ.get('game_data')
with open(char_path, 'r') as char_data:
    chars_json = json.load(char_data)

In [9]:
def fix_gear(dataf):
    g = 1
    c = 1
    length = dataf.shape[0]
    for i in range(0,length):
        if i % 6 == 0 and i != 0: g+=1
        dataf.at[i,'Gear'] = g
        dataf.at[i,'Item_order'] = c
        c+=1
        if c == 7: c = 1 
    return dataf


In [10]:
with sqlite3.connect(db_path) as conn:
    conn.execute('DROP TABLE IF EXISTS gear_items')
    for toon in chars_json:
        toon_data = chars_json[toon]['GearTiers']
        toon_df = pd.json_normalize(toon_data,record_path=['GearTierSlot'],errors='ignore')
        toon_df = fix_gear(toon_df)
        toon_df['Toon_ID'] = chars_json[toon]['ID']
        #if toon_df.shape[0] == 48:
        try:
            toon_df = toon_df[['Toon_ID', 'Gear', 'Item_order','Offense', 'Defense', 'Health', 'Recovery', 'CritChance', 'CritPower',
        'AttackSpeed','Speed', 'Potency', 'Tenacity',
        'Accuracy', 'Evasion','UnitLevel','ItemID']]
            toon_df.to_sql('gear_items',conn,if_exists='append',index=False)
        except:
            continue

In [4]:
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql('SELECT * FROM toons_categories', conn)
    df['category_name'] = df['category_name'].apply(lambda x: x.replace('UNIT_CATEGORY_','').replace('_',' ').title())
    df.to_sql('toons_categories',conn, if_exists='replace',index = False)

In [5]:
conn.commit()